In [2]:
import numpy as np
import time
from datetime import datetime
import copy
from tqdm import tqdm
import os
import matplotlib as mpl
from matplotlib import pyplot as plt

import importlib
import EnsembleLib
importlib.reload(EnsembleLib)
from EnsembleLib import time_diff

import auxFuncs
from auxFuncs import matrix_sqrt
from auxFuncs import makeCorrelatedMatrix
importlib.reload(auxFuncs)

import DatasetMaker
importlib.reload(DatasetMaker)
import Experiments
importlib.reload(Experiments)

from collections import OrderedDict

import torch
from torch.optim import SGD, Adam, lr_scheduler
device = 'cuda'

import pytz
from datetime import datetime
from dateutil.relativedelta import relativedelta
tz = pytz.timezone('US/Eastern')
from scipy.stats import ortho_group


def time_now():
    return datetime.now(tz)

def time_diff(t_a, t_b):
    t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
    return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

In [3]:
torch.set_default_dtype(torch.float64)

## For Figure 1: Power-Law correlations, feature noise, label noise, and postsynaptic noise.

In [8]:
#creates power law correlation matrix with nxn dimension and power law exponent -a
def plaw_matrix(n, a):
    row_vector = torch.arange(n)
    col_vector = torch.arange(n).reshape(-1, 1)
    diff = torch.abs(row_vector - col_vector)
    matrix = torch.pow(a, diff)
    return matrix

In [11]:
num_trials = 100
M = 2000
num_samples_list = np.round(np.linspace(M/20, M, 20))

lams = [1e-6, 1e-3, 1]
zeta = .1
eta = .2
expt_name = 'PLawCovTest'
sigma_s = plaw_matrix(M, .8).cuda()
sigma_0 = .1*plaw_matrix(M, .3).cuda()
nus = [.2, .4, .6]
ns = [int(nu*M) for nu in nus]
Experiments.runEvP_LinFixedMasks(expt_name, sigma_s, sigma_0, ns, eta, zeta, lams, num_samples_list, num_trials, aveGTW = False)

PLawCovTest
The experiment directory already exists!
P = 100.0Eg = 0.9624512825681779; time: 0h 0m 4s
P = 200.0Eg = 0.8512331456461847; time: 0h 0m 7s
P = 300.0Eg = 0.8382719643025162; time: 0h 0m 10s
P = 400.0Eg = 8.285527081684647; time: 0h 0m 13s
P = 500.0Eg = 0.7569538218654067; time: 0h 0m 16s
P = 600.0Eg = 0.6784387915532649; time: 0h 0m 19s
P = 700.0Eg = 0.7204817943767725; time: 0h 0m 21s
P = 800.0Eg = 5.58969965036387; time: 0h 0m 24s
P = 900.0Eg = 0.6816329910638059; time: 0h 0m 27s
P = 1000.0Eg = 0.6051080163638034; time: 0h 0m 30s
P = 1100.0Eg = 0.6325240143294415; time: 0h 0m 33s
P = 1200.0Eg = 3.2115587880128404; time: 0h 0m 35s
P = 1300.0Eg = 0.5799838554576844; time: 0h 0m 38s
P = 1400.0Eg = 0.4870176554106171; time: 0h 0m 38s
P = 1500.0Eg = 0.44001554723156666; time: 0h 0m 42s
P = 1600.0Eg = 0.41370617167634466; time: 0h 0m 44s
P = 1700.0Eg = 0.3934211457735318; time: 0h 0m 47s
P = 1800.0Eg = 0.3795455854119711; time: 0h 0m 49s
P = 1900.0Eg = 0.36877619192210903; time:

## For Figure 1: Global Correlations with 3 non-overlapping readouts, no feature noise

In [12]:
torch.set_default_dtype(torch.float64)

In [13]:
num_trials = 100
M = 5000
nus = [.1, .3, .5]
num_samples_list = np.round(np.linspace(M/20, M, 20))

lams = [1e-6, 1e-3, 1]
zeta = .1
eta = .1
s = 1
c = .4
omega = .1
expt_name = 'TripleGlobCorr'
sigma_s = auxFuncs.makeCorrelatedMatrix(M, c).to('cuda')
sigma_0 = omega*torch.eye(M, device = 'cuda')
rho = .3
ns = [int(round(nu*M)) for nu in nus]
Experiments.runEvP_LinFixedMasks(expt_name, sigma_s, sigma_0, ns, eta, zeta, lams, num_samples_list, num_trials, exclusive = True, aveGTW=True, rho = rho)

TripleGlobCorr
The experiment directory already exists!
P = 250.0Eg = 0.6924518312431854; time: 0h 2m 47s
P = 500.0Eg = 19.24757882762049; time: 0h 4m 28s
P = 750.0Eg = 0.747574955876669; time: 0h 8m 13s
P = 1000.0Eg = 0.6934966355639851; time: 0h 7m 56s
P = 1250.0Eg = 0.7771741935635003; time: 0h 4m 57s
P = 1500.0Eg = 10.390101477849319; time: 0h 5m 41s
P = 1750.0Eg = 0.8046835136713981; time: 0h 6m 16s
P = 2000.0Eg = 0.7156873061875024; time: 0h 6m 50s
P = 2250.0Eg = 0.7843992723816517; time: 0h 7m 23s
P = 2500.0Eg = 7.015516823769215; time: 0h 7m 55s
P = 2750.0Eg = 0.7573257541662658; time: 0h 7m 51s
P = 3000.0Eg = 0.620206633095007; time: 0h 8m 22s
P = 3250.0Eg = 0.5655441480129775; time: 0h 9m 5s
P = 3500.0Eg = 0.5338916621764108; time: 0h 9m 29s
P = 3750.0Eg = 0.5126108171777639; time: 0h 10m 10s
P = 4000.0Eg = 0.498577023989604; time: 0h 10m 31s
P = 4250.0Eg = 0.4867759349327034; time: 0h 10m 51s
P = 4500.0Eg = 0.47813990520238625; time: 0h 11m 22s
P = 4750.0Eg = 0.4697919090012

## For Figure 1: Global Correlations with 3 overlapping readouts, no feature noise

In [6]:
num_trials = 100
M = 5000
nus = [.1, .3, .5]
num_samples_list = np.round(np.linspace(M/20, M, 20))

lams = [1e-6, 1e-3, 1]
zeta = .1
eta = .1
omega = .1
c = .4
s = 1
expt_name = 'TripleGlobCorr_Overlap'
sigma_s = s*auxFuncs.makeCorrelatedMatrix(M, c).to('cuda')
sigma_0 = omega*torch.eye(M, device = 'cuda')
rho = 1
ns = [int(round(nu*M)) for nu in nus]
Experiments.runEvP_LinFixedMasks(expt_name, sigma_s, sigma_0, ns, eta, zeta, lams, num_samples_list, num_trials, exclusive = False, aveGTW=True, rho = rho)

TripleGlobCorr_Overlap
The experiment directory already exists!
P = 250.0Eg = 1.8699205424628864; time: 0h 2m 45s
P = 500.0Eg = 97.75527608458209; time: 0h 3m 20s
P = 750.0Eg = 2.2611139347187947; time: 0h 7m 37s
P = 1000.0Eg = 1.910172736898072; time: 0h 9m 12s
P = 1250.0Eg = 2.132948699860118; time: 0h 5m 9s
P = 1500.0Eg = 29.464656909580185; time: 0h 5m 20s
P = 1750.0Eg = 2.2051706517508234; time: 0h 5m 49s
P = 2000.0Eg = 1.9077564995943843; time: 0h 6m 19s
P = 2250.0Eg = 2.0130530919592515; time: 0h 6m 50s
P = 2500.0Eg = 14.786903460300286; time: 0h 7m 20s
P = 2750.0Eg = 1.930200237445506; time: 0h 7m 49s
P = 3000.0Eg = 1.638537921182357; time: 0h 8m 18s
P = 3250.0Eg = 1.5155538583138155; time: 0h 8m 47s
P = 3500.0Eg = 1.4472826555838678; time: 0h 9m 19s
P = 3750.0Eg = 1.3956786610170684; time: 0h 9m 46s
P = 4000.0Eg = 1.3608739948995214; time: 0h 10m 13s
P = 4250.0Eg = 1.3344980032531513; time: 0h 10m 42s
P = 4500.0Eg = 1.313165340595313; time: 0h 11m 14s
P = 4750.0Eg = 1.29494654

## For Figure 2: Subsampling Shifts the Double-Descent Peak

In [4]:
importlib.reload(Experiments)

<module 'Experiments' from '/n/home07/bruben/Simulations/Ensemble_Master/Experiments.py'>

In [6]:
num_trials = 100
M = 1000
nus = [.2, .4, .6, .8]
num_samples_list = np.round(np.linspace(M/20, M, 20)-5)
num_samples_list[-1] = num_samples_list[-1]+5

lams = [0, 1e-4, 1e-2]
zeta = 0
eta = 0
c = 0
expt_name = 'InterpThresh_GlobCorr'
sigma_s = auxFuncs.makeCorrelatedMatrix(M, c).to('cuda')
sigma_0 = torch.zeros(M,M, device = 'cuda')
ns = [int(round(nu*M)) for nu in nus]
Experiments.runEvP_SingleReadout_AlterThresh(expt_name, sigma_s, sigma_0, nus, eta, zeta, lams, num_samples_list, num_trials, aveGTW = True)

InterpThresh_GlobCorr
The experiment directory already exists!
P = 45.0; Eg = 1.1861365925875111; nu = 0.2; P = 45.0; time: 0h 0m 5s
P = 45.0; Eg = 1.0325119258687558; nu = 0.4; P = 45.0; time: 0h 0m 5s
P = 45.0; Eg = 0.9834963544897704; nu = 0.6; P = 45.0; time: 0h 0m 5s
P = 45.0; Eg = 0.9660272856563186; nu = 0.8; P = 45.0; time: 0h 0m 5s
P = 95.0; Eg = 1.6059785606318162; nu = 0.2; P = 95.0; time: 0h 0m 6s
P = 95.0; Eg = 1.0828154828332828; nu = 0.4; P = 95.0; time: 0h 0m 6s
P = 95.0; Eg = 0.9735287525747683; nu = 0.6; P = 95.0; time: 0h 0m 6s
P = 95.0; Eg = 0.9265549058560238; nu = 0.8; P = 95.0; time: 0h 0m 6s
P = 145.0; Eg = 2.9229047659650003; nu = 0.2; P = 145.0; time: 0h 0m 7s
P = 145.0; Eg = 1.1882952580722432; nu = 0.4; P = 145.0; time: 0h 0m 7s
P = 145.0; Eg = 0.9815061899107735; nu = 0.6; P = 145.0; time: 0h 0m 7s
P = 145.0; Eg = 0.8934643975089455; nu = 0.8; P = 145.0; time: 0h 0m 7s
P = 195.0; Eg = 24.203472113143924; nu = 0.2; P = 195.0; time: 0h 0m 8s
P = 195.0; Eg = 1

# Additional Stuff:

In [43]:
num_trials = 100
M = 1000
num_samples_list = np.round(np.linspace(M/10, M, 50))

lam = .001
zeta = 0
eta = 0
expt_name = 'TestLinEns'
sigma_s = torch.eye(M, device = 'cuda')
sigma_0 = torch.zeros(M,M, device = 'cuda')
#ns = [100, 200, 300]
ns = [100, 200, 300]
Experiments.runEvP_LinFixedMasks(expt_name, sigma_s, sigma_0, ns, eta, zeta, lam, num_samples_list, num_trials)

TestLinEns
The experiment directory already exists!
P = 100.0Eg = 2.627478173971176; time: 0h 0m 0s
P = 118.0Eg = 1.6407646644115448; time: 0h 0m 0s
P = 137.0Eg = 1.4319559776782989; time: 0h 0m 0s
P = 155.0Eg = 1.453303072452545; time: 0h 0m 0s
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/n/home07/bruben/.conda/envs/ffcv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_47049/3159816335.py", line 13, in <cell line: 13>
    Experiments.runEvP_LinFixedMasks(expt_name, sigma_s, sigma_0, ns, eta, zeta, lam, num_samples_list, num_trials)
  File "/n/home07/bruben/Simulations/Ensemble_Master/Experiments.py", line 256, in runEvP_LinFixedMasks
    student.regress_readout(train_loader, test_loader, lam)
  File "/n/home07/bruben/Simulations/Ensemble_Master/EnsembleLib.py", line 238, in regress_readout
    W = psi@torch.linalg.solve(psiT@psi + lam*torch.eye(P, device = 'cuda'), Y)#torch.linalg.inv(psiT@psi + lam*torch.eye(P, device = 'cuda'))@Y
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/n/home07/bruben/.conda/envs/ffcv/lib/python3.9/site-pa

Compare homogeneous and heterogeneous on correlated gaussian data

In [36]:
M = 1000


expt_name = 'UnifVsHet_Lin_Corr'
sigma_s = auxFuncs.makeCorrelatedMatrix(M, .5).to('cuda')
sigma_0 = torch.zeros(M,M, device = 'cuda')
connDists = ['delta', 'gauss']
KVals = [2, 4, 8, 16]
zeta = 0
eta = 0
lams = [.001, 1]
num_samples_list = np.round(np.linspace(M/20, 1.5*M, 200))
num_trials=100

Experiments.runEvP_Lin_CompDists(expt_name, sigma_s, sigma_0, connDists, KVals, eta, zeta, lams, num_samples_list, num_trials)

UnifVsHet_Lin_Corr
The experiment directory already exists!
P = 50.0; Eg = 0.494394907951355; dist = delta; K = 2; lam = 0.001; time: 0h 0m 0s
P = 50.0; Eg = 0.49162455290555956; dist = delta; K = 2; lam = 1; time: 0h 0m 0s
P = 57.0; Eg = 0.48916320770978927; dist = delta; K = 2; lam = 0.001; time: 0h 0m 0s
P = 57.0; Eg = 0.49059606850147247; dist = delta; K = 2; lam = 1; time: 0h 0m 0s
P = 65.0; Eg = 0.48823714882135394; dist = delta; K = 2; lam = 0.001; time: 0h 0m 0s
P = 65.0; Eg = 0.48821416109800336; dist = delta; K = 2; lam = 1; time: 0h 0m 0s
P = 72.0; Eg = 0.4872340524196625; dist = delta; K = 2; lam = 0.001; time: 0h 0m 0s
P = 72.0; Eg = 0.4833978402614594; dist = delta; K = 2; lam = 1; time: 0h 0m 0s
P = 79.0; Eg = 0.4846160414814949; dist = delta; K = 2; lam = 0.001; time: 0h 0m 0s
P = 79.0; Eg = 0.48114356070756914; dist = delta; K = 2; lam = 1; time: 0h 0m 0s
P = 86.0; Eg = 0.48311561346054077; dist = delta; K = 2; lam = 0.001; time: 0h 0m 0s
P = 86.0; Eg = 0.4793746370077

# Single Readout with Correlations

In [7]:
num_trials = 10
M = 2000
num_samples_list = np.round(np.linspace(M/10, M, 50))

lam = .1
zeta = 0
eta = 0
expt_name = 'SingleGlobCorr'
c=.2
sigma_s = auxFuncs.makeCorrelatedMatrix(M, c).to('cuda')
sigma_0 = torch.zeros(M,M, device = 'cuda')
ns = [500]
Experiments.runEvP_LinFixedMasks(expt_name, sigma_s, sigma_0, ns, eta, zeta, lam, num_samples_list, num_trials)

SingleGlobCorr
The experiment directory already exists!
P = 200.0Eg = 0.963396567106247; time: 0h 0m 0s
P = 237.0Eg = 1.0186649322509767; time: 0h 0m 0s
P = 273.0Eg = 1.0653776168823241; time: 0h 0m 0s
P = 310.0Eg = 1.0921717882156372; time: 0h 0m 0s
P = 347.0Eg = 1.156273567676544; time: 0h 0m 0s
P = 384.0Eg = 1.1639367818832398; time: 0h 0m 0s
P = 420.0Eg = 1.2159821271896363; time: 0h 0m 0s
P = 457.0Eg = 1.238257372379303; time: 0h 0m 0s
P = 494.0Eg = 1.2267664074897766; time: 0h 0m 0s
P = 531.0Eg = 1.2467477440834045; time: 0h 0m 0s
P = 567.0Eg = 1.2508822441101075; time: 0h 0m 0s
P = 604.0Eg = 1.2316078186035155; time: 0h 0m 0s
P = 641.0Eg = 1.1996636986732483; time: 0h 0m 0s
P = 678.0Eg = 1.2250660061836243; time: 0h 0m 0s
P = 714.0Eg = 1.186125934123993; time: 0h 0m 0s
P = 751.0Eg = 1.1915929555892943; time: 0h 0m 0s
P = 788.0Eg = 1.143102478981018; time: 0h 0m 0s
P = 824.0Eg = 1.1299226522445678; time: 0h 0m 0s
P = 861.0Eg = 1.1142508029937743; time: 0h 0m 0s
P = 898.0Eg = 1.09

# Two Readouts with global correlation and no overlap

In [58]:
num_trials = 10
M = 1000
nur = .2,
nurp = .6
num_samples_list = np.round(np.linspace(M/10, M, 20))

lams = [0, 1e-6, 1e-3, 1]
zeta = .1
eta = 0.2
c = .4
expt_name = 'DoubleGlobCorr'
sigma_s = auxFuncs.makeCorrelatedMatrix(M, c).to('cuda')
sigma_0 = torch.zeros(M,M, device = 'cuda')
ns = [int(round(nur*M)), int(round(nurp*M))]
Experiments.runEvP_LinFixedMasks(expt_name, sigma_s, sigma_0, ns, eta, zeta, lams, num_samples_list, num_trials, exclusive = True)

DoubleGlobCorr
The experiment directory already exists!
P = 100.0Eg = 0.7245773792266845; time: 0h 0m 0s
P = 147.0Eg = 0.745173555612564; time: 0h 0m 0s
P = 195.0Eg = 0.8259103178977967; time: 0h 0m 0s
P = 242.0Eg = 1.084385883808136; time: 0h 0m 0s
P = 289.0Eg = 4.811034226417542; time: 0h 0m 0s
P = 337.0Eg = 1.5346452355384828; time: 0h 0m 0s
P = 384.0Eg = 0.9564656257629395; time: 0h 0m 0s
P = 432.0Eg = 0.8698632121086121; time: 0h 0m 0s
P = 479.0Eg = 0.8248958647251129; time: 0h 0m 0s
P = 526.0Eg = 0.9452146828174591; time: 0h 0m 0s
P = 574.0Eg = 1.965213131904602; time: 0h 0m 0s
P = 621.0Eg = 2.276796340942383; time: 0h 0m 0s
P = 668.0Eg = 1.0275730788707733; time: 0h 0m 0s
P = 716.0Eg = 0.6967135369777679; time: 0h 0m 0s
P = 763.0Eg = 0.576133382320404; time: 0h 0m 0s
P = 811.0Eg = 0.5196708798408508; time: 0h 0m 0s
P = 858.0Eg = 0.4792963206768036; time: 0h 0m 0s
P = 905.0Eg = 0.4526291787624359; time: 0h 0m 0s
P = 953.0Eg = 0.4313360720872879; time: 0h 0m 0s
P = 1000.0Eg = 0.42